In [ ]:
# Controls
# wz_wz_flag = 0

In [ ]:
import pandas as pd
import numpy as np
import geopy.distance
import ast
import matplotlib.pyplot as plt
import random
import math
import time

In [ ]:
# Perceived Difficulty Data per Route Zone
rz_pd = pd.read_csv('../PD 1.5/pd_1.5_agg.csv')

In [ ]:
rz_pd['new_perceived_difficulty_total'] = rz_pd['pd_total']
rz_pd['new_perceived_difficulty_within'] = rz_pd['pd_within_small'] + rz_pd['pd_within_bulky']
rz_pd['new_perceived_difficulty_across'] = rz_pd['pd_across_short_small'] + rz_pd['pd_across_long_small'] + rz_pd['pd_across_short_bulky'] + rz_pd['pd_across_long_bulky']
rz_pd['new_perceived_difficulty_across_short'] = rz_pd['pd_across_short_small'] + rz_pd['pd_across_short_bulky']
rz_pd['new_perceived_difficulty_across_long'] = rz_pd['pd_across_long_small'] + rz_pd['pd_across_long_bulky']
rz_pd['new_perceived_difficulty_addons'] = rz_pd['pd_rpu'] + rz_pd['pd_cod']

In [ ]:
# List of Route Zones within proximity of Base Route Zone
rz_proximity = pd.read_csv('../Proximity 1.5/proximity_1.5.csv') #
route_zones = rz_proximity[['route_zone','route_zone_short','route_zone_centroid_lat', 'route_zone_centroid_long']] #

# Route Zone and Station Lat Long
rz_station = pd.read_csv('station_route_zones.csv') #

# Volume and Estimate Number of Trips Data
vol_route_zone_level = pd.read_csv('rz_avg_vol_and_trips.csv')

# Route zone combination limiter
rz_limiter = pd.read_csv('rz_limiter.csv')

# Route zone pre-merge list
non_nego_rz = pd.read_csv('non_nego_merged_routes.csv')

rz_small_bulky_ratio = rz_pd[['route_zone','route_zone_og','small_vol','bulky_vol']]

In [ ]:
rz_pd['route_zone'] = rz_pd['route_zone'].str.strip()
route_zones['route_zone'] = route_zones['route_zone'].str.strip()
rz_station['route_zone'] = rz_station['route_zone'].str.strip()
vol_route_zone_level['route_zone'] = vol_route_zone_level['route_zone'].str.strip()

rz_limiter['route_zone'] = rz_limiter['route_zone'].str.strip()
non_nego_rz['route_zone'] = non_nego_rz['route_zone'].str.strip()
non_nego_rz['route_zone_trimmed'] = non_nego_rz['route_zone_trimmed'].str.strip()

In [ ]:
# vol_route_zone_level
route_zone = []
daily_avg_bulky = []
daily_avg_small = []
daily_avg_vol = []
daily_calculated_avg_vol = []
daily_avg_no_of_trips = []

for i in range(0,len(vol_route_zone_level)):
    temp = vol_route_zone_level.iloc[i]

    rz = temp['route_zone'].strip()
    
    temp2 = rz_small_bulky_ratio.loc[rz_small_bulky_ratio['route_zone_og']==rz]

    if len(temp2) != 2:
        print(rz)
    
    walker = temp2.loc[temp2['route_zone'].str.contains('walker')]
    driver = temp2.loc[temp2['route_zone'].str.contains('driver')]
        
    small_walker = walker['small_vol'].values[0]
    small_driver = driver['small_vol'].values[0]

    bulky_walker = walker['bulky_vol'].values[0]
    bulky_driver = driver['bulky_vol'].values[0]

    route_zone.append(rz+'_driver')
    route_zone.append(rz+'_walker')
    
    # bulky
    if bulky_walker+bulky_driver == 0:
        daily_avg_bulky.append(0)
        daily_avg_bulky.append(0)
    else:
        daily_avg_bulky.append( (bulky_driver/(bulky_walker+bulky_driver)) * temp['daily_avg_bulky']) # driver
        daily_avg_bulky.append( (bulky_walker/(bulky_walker+bulky_driver)) * temp['daily_avg_bulky']) # walker

    # small
    if small_walker+small_driver == 0:
        daily_avg_small.append(0)
        daily_avg_small.append(0)
    else:
        daily_avg_small.append( (small_driver/(small_walker+small_driver)) * temp['daily_avg_small'])
        daily_avg_small.append( (small_walker/(small_walker+small_driver)) * temp['daily_avg_small'])

In [ ]:
vol_route_zone_level2 = pd.DataFrame({'route_zone':route_zone,'daily_avg_small':daily_avg_small,'daily_avg_bulky':daily_avg_bulky})
vol_route_zone_level2['daily_avg_vol'] = vol_route_zone_level2['daily_avg_small'] + vol_route_zone_level2['daily_avg_bulky']
vol_route_zone_level2['daily_calculated_avg_vol'] = vol_route_zone_level2['daily_avg_small'] + (vol_route_zone_level2['daily_avg_bulky']*6)
vol_route_zone_level2['daily_avg_no_of_trips'] = vol_route_zone_level2['daily_calculated_avg_vol']/540

In [ ]:
### New Cleaning Code
route_zone = []
station_name = [] 
station_lat = []
station_long = []

for i in range(0,len(rz_station)):
    temp = rz_station.iloc[i]
    route_zone.append(temp['route_zone']+'_driver')
    route_zone.append(temp['route_zone']+'_walker')
    
    station_name.append(temp['station_name'])
    station_name.append(temp['station_name'])
    
    station_lat.append(temp['station_lat'])
    station_lat.append(temp['station_lat'])
    
    station_long.append(temp['station_long'])
    station_long.append(temp['station_long'])

rz_station2 = pd.DataFrame({'route_zone':route_zone,'station_name':station_name,'station_lat':station_lat,'station_long':station_long})
###

In [ ]:
route_zone = []
rl = []
for i in range(0,len(rz_limiter)):
    temp = rz_limiter.iloc[i]
    temp2 = []

    rz = temp['route_zone'].strip()
    route_zone.append(rz+'_driver')
    route_zone.append(rz+'_walker')

    try:
        rz_list = ast.literal_eval(temp['rz_limiter'])
    except:
        rz_list = []

    for j in rz_list:
        temp2.append(j+'_driver')
        temp2.append(j+'_walker')
    rl.append(temp2)
    rl.append(temp2)

rz_limiter2 = pd.DataFrame({'route_zone':route_zone,'rz_limiter':rl})

In [ ]:
non_nego_rz.head()

In [ ]:
# Input Constraints
rz_proximity_control = 0.5
PD_control = 10

boosted_rz_proximity_control = 2 # 1st Boost Parameter for proximity control
boosted_PD_control = 7 # 1st Boost Parameter for PD control

second_boosted_rz_proximity_control = 5 # 2nd Boost Parameter for proximity control
second_boosted_PD_control = 8

working_hour_boost_val = 7 # 1st Boost - If combined zone result in less than this number of supply hours, re-run calculation for those zones
second_working_hour_boost_val = 7 # 2nd Boost - If combined zone result in less than this number of supply hours, re-run calculation for those zones

route_collect_time_per_parcel = 0.002556 #9.2s of route collect time per parcel
avg_driving_time = 50 #km /hour
vehicle_calculated_capacity = 540 # Number of equivalent small parcels a vehicle can carry

In [ ]:
# Inputs for COS Calculation
pay_per_hour = 15 # Hourly pay per hour 
driver_fixed_cost = 2200 # Monthly fixed cost per driver
working_days = 25 # Number of working days

# Number of iterations to run
num_of_iterations = 2

In [ ]:
vol_route_zone_level2

In [ ]:
# Average volumes per Route Zone Data
# vol_route_zone_level2

rz_proximity2 = route_zones.merge(rz_pd, on='route_zone', how='left')

rz_proximity3 = rz_proximity2.merge(vol_route_zone_level2[['route_zone', 'daily_avg_bulky', 'daily_avg_small', 'daily_avg_vol', 'daily_calculated_avg_vol', 'daily_avg_no_of_trips']], on='route_zone', how='left')

rz_proximity3 = rz_proximity3.merge(rz_station[['route_zone','station_name','station_lat','station_long']], on='route_zone', how='left')

# Remove route zones with no daily avg vol
rz_proximity3_clean = rz_proximity3.dropna().sort_values(by='daily_avg_vol').reset_index(drop=True)

# Remove route zones with no calculated PD
rz_proximity3_clean = rz_proximity3.dropna().sort_values(by='new_perceived_difficulty_total').reset_index(drop=True)

raw_data = rz_proximity3_clean

In [ ]:
# Concatenating the non-null variables into a string/"list" - eg. 'A01-A','A01-B'
non_nego_rz['rz_non_negotiables'] = non_nego_rz[['route_zone_1', 'route_zone_2', 'route_zone_3','route_zone_4','route_zone_5',
                 'route_zone_6', 'route_zone_7', 'route_zone_8','route_zone_9','route_zone_10',
                 'route_zone_11', 'route_zone_12', 'route_zone_13','route_zone_14','route_zone_15',
                ]].apply(lambda x: '['+",".join(x.dropna())+']', axis=1)

In [ ]:
non_nego_rz = non_nego_rz[non_nego_rz['rz_non_negotiables']!="[]"].reset_index(drop=True)

In [ ]:
route_zone = []
count_of_mergeable_zones = [] 
rz_non_negotiables = []

for i in range(0,len(non_nego_rz)):
    temp = non_nego_rz.iloc[i]

    rnn = []
    
    rz = temp['route_zone'].strip()
    route_zone.append(rz.strip()+'_driver')
    route_zone.append(rz.strip()+'_walker')

    count_of_mergeable_zones.append(temp['count_of_mergeable_zones']*2)
    count_of_mergeable_zones.append(temp['count_of_mergeable_zones']*2)
    
    for j in ast.literal_eval(temp['rz_non_negotiables']):
        rnn.append(j.strip()+'_driver')
        rnn.append(j.strip()+'_walker')
        
    rz_non_negotiables.append(rnn)
    rz_non_negotiables.append(rnn)

In [ ]:
non_nego_rz2 = pd.DataFrame({'route_zone':route_zone,'count_of_mergeable_zones':count_of_mergeable_zones,'rz_non_negotiables':rz_non_negotiables})

In [ ]:
raw_premerged = raw_data.merge(non_nego_rz2[['route_zone', 'rz_non_negotiables']],on='route_zone', how='left')
raw_premerged ['rz_non_negotiables'] = raw_premerged ['rz_non_negotiables'].fillna(0)
raw_premerged ['pre_merge_flag'] = 0

In [ ]:
raw_premerged.head()

In [ ]:
# Pre-addition of metrics for pre-merged zones

for i in range(0, len(raw_premerged)):

    temp_rz = raw_premerged['route_zone'].iloc[i]

    # If there are no pre-merges required, move on
    if not raw_premerged['rz_non_negotiables'].iloc[i]:
        continue

    # If zone is already merged, move on
    if raw_premerged['pre_merge_flag'].iloc[i] == 1:
        continue

    total_num_zones = 1

    final_lat = raw_premerged['route_zone_centroid_lat'].iloc[i]
    final_long = raw_premerged['route_zone_centroid_long'].iloc[i]

    for premerge_zone in ast.literal_eval(str(raw_premerged['rz_non_negotiables'].iloc[i])):

        if raw_premerged[raw_premerged['route_zone'] == premerge_zone]['pre_merge_flag'].iloc[0] == 1:
            continue

        total_num_zones += 1

        # Mark zone as pre-merged
        raw_premerged.loc[raw_premerged['route_zone'] == premerge_zone, 'pre_merge_flag'] = 1

        temp_premerge = raw_premerged[raw_premerged['route_zone'] == premerge_zone]

        # Add on values for pre-merged zone

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'new_perceived_difficulty_total'] += temp_premerge['new_perceived_difficulty_total'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'new_perceived_difficulty_within'] += temp_premerge['new_perceived_difficulty_within'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'new_perceived_difficulty_across'] += temp_premerge['new_perceived_difficulty_across'].iloc[0]
        
        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'new_perceived_difficulty_addons'] += temp_premerge['new_perceived_difficulty_addons'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'daily_avg_bulky'] += temp_premerge['daily_avg_bulky'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'daily_avg_small'] += temp_premerge['daily_avg_small'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'daily_avg_vol'] += temp_premerge['daily_avg_vol'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'daily_calculated_avg_vol'] += temp_premerge['daily_calculated_avg_vol'].iloc[0]

        raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'daily_avg_no_of_trips'] += temp_premerge['daily_avg_no_of_trips'].iloc[0]

        # raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'avg_daily_wp'] += temp_premerge['avg_daily_wp'].iloc[0]

        final_lat += temp_premerge['route_zone_centroid_lat'].iloc[0]

        final_long += temp_premerge['route_zone_centroid_long'].iloc[0]

    # Update new centroids for pre-merged zones

    raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'route_zone_centroid_lat'] = final_lat / total_num_zones
    raw_premerged.loc[raw_premerged['route_zone'] == temp_rz, 'route_zone_centroid_long'] = final_long / total_num_zones

In [ ]:
raw_premerged[raw_premerged['route_zone'] == 'F01-B_driver']

In [ ]:
len(raw_premerged)

In [ ]:
# Store list of non-negotiable zone merges
premerge_list = raw_premerged[(raw_premerged['pre_merge_flag'] == 0) & (raw_premerged['rz_non_negotiables'] != False)][['route_zone', 'rz_non_negotiables']]

In [ ]:
len(premerge_list)

In [ ]:
final_premerged = raw_premerged[raw_premerged['pre_merge_flag'] == 0]

In [ ]:
len(final_premerged)

In [ ]:
final_premerged.head(2)

In [ ]:
rz_proximity = []
rz_proximity_distance = []

for i in range(0,len(final_premerged['route_zone'])):
    temp = final_premerged.iloc[i]

    main_lat = temp['route_zone_centroid_lat']
    main_long = temp['route_zone_centroid_long']

    closest_rz = []
    closest_rz_distance = []

    for j in range(0,len(final_premerged['route_zone'])):
        temp2 = final_premerged.iloc[j]

        second_lat = temp2['route_zone_centroid_lat']
        second_long = temp2['route_zone_centroid_long']

        dist = geopy.distance.distance((main_lat,main_long),(second_lat,second_long)).km
        if dist <= 10 and dist > 0:
            closest_rz.append(temp2['route_zone'])
            closest_rz_distance.append(dist)

    rz_proximity.append([closest_rz for _, closest_rz in sorted(zip(closest_rz_distance,closest_rz))])
#     print(sorted(zip(closest_rz_distance,closest_rz)))
    closest_rz_distance.sort()

    rz_proximity_distance.append(closest_rz_distance)

In [ ]:
final_premerged['rz_proximity'] = rz_proximity
final_premerged['rz_proximity_distance'] = rz_proximity_distance

In [ ]:
final_premerged[final_premerged['route_zone'] == 'A01-F_walker']

In [ ]:
final_copy = final_premerged.copy()

In [ ]:
len(final_copy)

In [ ]:
rz_proximity_check = final_premerged.merge(rz_limiter2, how ='inner', on ='route_zone')
rz_proximity_check2 = rz_proximity_check.dropna()
rz_proximity_check2 = rz_proximity_check2[['route_zone','rz_proximity','rz_proximity_distance','rz_limiter']]

In [ ]:
rz_limiter2

In [ ]:
rz_proximity_check2.head(5)

In [ ]:
final_rz_proximity = []
final_rz_proximity_dist = []

for i in range(0,len(rz_proximity_check2)):

    temp = rz_proximity_check2.iloc[i]

# Convert rz_proximity, rz_limiter and rz_proximity_distance into a list
    rz_p = ast.literal_eval(str(temp['rz_proximity']))
    rz_l = ast.literal_eval(str(temp['rz_limiter']))
    rz_p_d = ast.literal_eval(str(temp['rz_proximity_distance']))

    new_proximity_list = []
    new_distance_list = []

# For each route zone in proximity
    for j in range(0,len(rz_p)):

        if rz_p[j] in rz_l: # If it is found in the limiter, ignore the route zone
            continue

        new_proximity_list.append(rz_p[j])
        new_distance_list.append(rz_p_d[j])

    final_rz_proximity.append(new_proximity_list)
    final_rz_proximity_dist.append(new_distance_list)

rz_proximity_check2['new_rz_proximity_list'] = final_rz_proximity
rz_proximity_check2['new_rz_proximity_dist'] = final_rz_proximity_dist

In [ ]:
rz_proximity_check2[rz_proximity_check2['route_zone'] == 'A01-F_driver']

In [ ]:
# Construct new rz_proximity table with updated rz_proximity list and rz_proximity_dist lists
final_premerged_limited = final_premerged.merge(rz_proximity_check2[['route_zone', 'new_rz_proximity_list', 'new_rz_proximity_dist']], how = 'left', left_on = 'route_zone', right_on ='route_zone')
final_premerged_limited = final_premerged_limited.fillna(0)

In [ ]:
rz_proximity = []
rz_proximity_list = []

for i in range(0,len(final_premerged_limited)):
    temp = final_premerged_limited.iloc[i]

    if not temp['new_rz_proximity_list']:
        rz_proximity.append(temp['rz_proximity'])
        rz_proximity_list.append(temp['rz_proximity_distance'])

    else:
        rz_proximity.append(temp['new_rz_proximity_list'])
        rz_proximity_list.append(temp['new_rz_proximity_dist'])

final_premerged_limited['final_rz_proximity'] = rz_proximity
final_premerged_limited['final_rz_proximity_dist'] = rz_proximity_list

In [ ]:
final = final_premerged_limited.drop(['rz_proximity', 'rz_proximity_distance', 'new_rz_proximity_list', 'new_rz_proximity_dist'], axis=1)

In [ ]:
final = final.rename({'final_rz_proximity': 'rz_proximity', 'final_rz_proximity_dist': 'rz_proximity_distance'}, axis=1)

In [ ]:
final[final['route_zone'] == 'A01-F_driver']

In [ ]:
len(final)

In [ ]:
# Data Transformation

# Function to calculate station - route centroid distance and station - route perceived difficulty
# NOTE: This calculation includes 2 trips; to and from between station and route
def station_route_pd(station_lat, station_long, route_zone_centroid_lat, route_zone_centroid_long, avg_driving_time):

    # Includes collect and return trips
    dist = geopy.distance.distance((station_lat, station_long),(route_zone_centroid_lat,route_zone_centroid_long)).km * 2
    station_route_pd = dist / avg_driving_time

    return station_route_pd

# Function to calculate route collect time = route collect time per parcel * average daily volume (mixed size)
def route_collect_time(route_collect_time_per_parcel, daily_avg_vol):

    route_collect_time = route_collect_time_per_parcel*daily_avg_vol

    return route_collect_time


# Calculate station to route travel time per route zone
final['station_to_route_pd_per_trip'] = final.apply(lambda x: station_route_pd(x['station_lat'], x['station_long'], x['route_zone_centroid_lat'], x['route_zone_centroid_long'], avg_driving_time), axis=1)

# Calculate route collection time per route zone
final['route_collect_pd'] = final.apply(lambda x: route_collect_time(route_collect_time_per_parcel, x['daily_avg_vol']), axis=1)


In [ ]:
len(final)

In [ ]:
# Function for route combiner algorithm

def route_combiner_algorithm(input_dataset, rz_proximity_control, PD_control, vehicle_calculated_capacity, avg_driving_time, random_state_route_zone):

    base = []

    combination = []
    combination_pd = []
    combination_pd_within = []
    combination_pd_across = []
    combination_pd_addons = []
    route_collect_pd = []
    station_route_pd = []
    inter_zone_pd = []

    combination_pd_breakdown = []
    daily_avg_vol_breakdown = []
    num_trips_breakdown = []
    # num_waypoints_breakdown = []

    daily_calculated_vol = []
    daily_avg_small = []
    daily_avg_bulky = []
    daily_avg_vol = []

    # daily_avg_wp = []

    num_trips = []



    been_combined = [] # list to check which route zones have been combined

    final = input_dataset.sample(frac = 1, random_state = random_state_route_zone).reset_index(drop=True).copy()

    for i in range(0, len(final)):
        temp = final.iloc[i]

        if temp['route_zone'] in been_combined:
            continue
        been_combined.append(temp['route_zone'].strip())
        
        # Extract current base zone metrics

        combo_zone = [temp['route_zone'].strip()]

        combo_pd_within = temp['new_perceived_difficulty_within']
        combo_pd_across = temp['new_perceived_difficulty_across']
        combo_pd_addons = temp['new_perceived_difficulty_addons']
        combo_route_collect_pd = temp['route_collect_pd']
        base_station_route_pd_per_trip = temp['station_to_route_pd_per_trip']

        dict_combination_pd_breakdown = dict()
        dict_daily_avg_vol_breakdown = dict()
        dict_num_trips_breakdown = dict()
        # dict_num_waypoints_breakdown = dict()

        dict_combination_pd_breakdown[temp['route_zone'].strip()] = round((combo_pd_within + combo_pd_across + combo_pd_addons), 1)
        dict_daily_avg_vol_breakdown[temp['route_zone'].strip()] = round(temp['daily_avg_vol'], 1)
        dict_num_trips_breakdown[temp['route_zone'].strip()] = round((temp['daily_calculated_avg_vol'] / vehicle_calculated_capacity), 3)
        # dict_num_waypoints_breakdown[temp['route_zone'].strip()] = round(temp['avg_daily_wp'], 1)

        combo_calculated_vol = temp['daily_calculated_avg_vol']
        combo_small_vol = temp['daily_avg_small']
        combo_bulky_vol = temp['daily_avg_bulky']
        combo_total_vol = combo_small_vol + combo_bulky_vol

        # combo_total_wp = temp['avg_daily_wp']

        combo_inter_route_zone_pd = 0

        base.append(temp['route_zone'].strip())

        rz_p = ast.literal_eval(str(temp['rz_proximity']))
        rz_p_d = ast.literal_eval(str(temp['rz_proximity_distance']))

        seq = list(range(0,len(rz_p)))

        # Go through list of zones that could be combined with base route zone

        for j in seq:
            temp_rz = rz_p[j]

            estimated_num_trips = math.ceil(combo_calculated_vol / vehicle_calculated_capacity)

            # Check if existing route zone meets supply hour constraint
            if combo_pd_within + combo_pd_across + combo_pd_addons + combo_route_collect_pd + (base_station_route_pd_per_trip*estimated_num_trips) +\
            (combo_inter_route_zone_pd*estimated_num_trips) >= PD_control:
                break

            # Check if proximity route zone has already been combined
            if temp_rz in been_combined: # If the route zone had already been combined, it will not be combined
                continue

            # Check if proximity route zone meets radius constraint
            if rz_p_d[j] > rz_proximity_control: # If the route zone is above the radius constraint, it will not be combined
                continue

            if len(final[final['route_zone']==temp_rz.strip()]) == 0:
                continue
                
            # Combined zone metrics

            temp_pd_within = final[final['route_zone']==temp_rz.strip()]['new_perceived_difficulty_within'].iloc[0]
            temp_pd_across = final[final['route_zone']==temp_rz.strip()]['new_perceived_difficulty_across'].iloc[0]
            temp_pd_addons = final[final['route_zone']==temp_rz.strip()]['new_perceived_difficulty_addons'].iloc[0]
            temp_route_collect_pd = final[final['route_zone']==temp_rz.strip()]['route_collect_pd'].iloc[0]
            temp_inter_route_zone_pd = rz_p_d[j]/avg_driving_time

            temp_calculated_vol = final[final['route_zone']==temp_rz.strip()]['daily_calculated_avg_vol'].iloc[0]
            temp_small_vol = final[final['route_zone']==temp_rz.strip()]['daily_avg_small'].iloc[0]
            temp_bulky_vol = final[final['route_zone']==temp_rz.strip()]['daily_avg_bulky'].iloc[0]
            temp_total_vol = temp_small_vol + temp_bulky_vol

            # temp_total_wp = final[final['route_zone']==temp_rz.strip()]['avg_daily_wp'].iloc[0]

            estimated_num_trips = math.ceil((combo_calculated_vol + temp_calculated_vol) / vehicle_calculated_capacity)

            temp_combo_pd = combo_pd_within + combo_pd_across + combo_pd_addons + combo_route_collect_pd +\
                            temp_pd_within + temp_pd_across + temp_pd_addons + temp_route_collect_pd +\
                            (base_station_route_pd_per_trip*estimated_num_trips) +\
                            (temp_inter_route_zone_pd*estimated_num_trips) +\
                            (combo_inter_route_zone_pd*estimated_num_trips)

            if temp_combo_pd >= PD_control: # If the combined route zone's total PD is not within the PD constraint, do not combine
                continue

            combo_pd_within += temp_pd_within # If the route zone meets all constraints, proceed to combine
            combo_pd_across += temp_pd_across
            combo_pd_addons += temp_pd_addons
            combo_route_collect_pd += temp_route_collect_pd
            combo_inter_route_zone_pd += temp_inter_route_zone_pd

            combo_calculated_vol += temp_calculated_vol
            combo_small_vol += temp_small_vol
            combo_bulky_vol += temp_bulky_vol
            combo_total_vol += temp_total_vol

            # combo_total_wp += temp_total_wp

            combo_zone.append(temp_rz)

            dict_combination_pd_breakdown[temp_rz] = round((temp_pd_within + temp_pd_across + temp_pd_addons), 1)
            dict_daily_avg_vol_breakdown[temp_rz] = round(temp_total_vol, 1)
            dict_num_trips_breakdown[temp_rz] = round((temp_calculated_vol / vehicle_calculated_capacity), 3)
            # dict_num_waypoints_breakdown[temp_rz] = round(temp_total_wp, 1)

            been_combined.append(temp_rz) # Add the newly combined route zone to been_combined list


        final_num_trips = math.ceil(combo_calculated_vol / vehicle_calculated_capacity)

        combo_pd = combo_pd_within + combo_pd_across + combo_pd_addons + combo_route_collect_pd +\
                    (combo_inter_route_zone_pd*final_num_trips) + (base_station_route_pd_per_trip*final_num_trips)

        combination.append(combo_zone)
        combination_pd.append(combo_pd)

        combination_pd_breakdown.append(dict_combination_pd_breakdown)
        daily_avg_vol_breakdown.append(dict_daily_avg_vol_breakdown)
        num_trips_breakdown.append(dict_num_trips_breakdown)
        # num_waypoints_breakdown.append(dict_num_waypoints_breakdown)

        combination_pd_within.append(combo_pd_within)
        combination_pd_across.append(combo_pd_across)
        combination_pd_addons.append(combo_pd_addons)
        route_collect_pd.append(combo_route_collect_pd)
        inter_zone_pd.append(combo_inter_route_zone_pd*final_num_trips)
        station_route_pd.append(base_station_route_pd_per_trip*final_num_trips)

        daily_calculated_vol.append(combo_calculated_vol)
        daily_avg_small.append(combo_small_vol)
        daily_avg_bulky.append(combo_bulky_vol)
        daily_avg_vol.append(combo_total_vol)

        # daily_avg_wp.append(combo_total_wp)

        num_trips.append(final_num_trips)

    output = pd.DataFrame({'base':base,
                            'combination':combination,
                            'combination_pd_breakdown':combination_pd_breakdown,
                            'daily_calculated_vol_breakdown':daily_avg_vol_breakdown,
                            'num_trips_breakdown':num_trips_breakdown,
                            # 'num_waypoints_breakdown':num_waypoints_breakdown,
                            'est_working_hours':combination_pd,
                            'delivery_time_within':combination_pd_within,
                            'delivery_time_across':combination_pd_across,
                            'delivery_time_addons':combination_pd_addons,
                            'route_collect_time':route_collect_pd,
                            'inter_zone_pd':inter_zone_pd,
                            'station_route_travel_time':station_route_pd,
                            'number_of_trips':num_trips,
                            'daily_calculated_vol':daily_calculated_vol,
                            'daily_avg_small':daily_avg_small,
                            'daily_avg_bulky':daily_avg_bulky,
                            'daily_avg_vol':daily_avg_vol
                            # 'daily_avg_waypoints':daily_avg_wp

                })

    return output, random_state_route_zone

In [ ]:
# Loop to run iterations

df_list = []
df_name_list = []

for iteration in range(0, num_of_iterations):

    start_time = time.time()
    random_state_rz_integer= random.randrange(9999)

    output_df, random_state_rz = route_combiner_algorithm(final, rz_proximity_control, PD_control, vehicle_calculated_capacity, avg_driving_time, random_state_rz_integer)

    df_list.append(output_df)
    df_name_list.append(f'Scenario_{iteration + 1}_{random_state_rz_integer}')

    print(f"Iteration {iteration + 1} Done")
    print("--- {} seconds ---".format(time.time() - start_time))

In [ ]:
# Check that all zones combined are accounted for

for i in range(0,len(df_list)):
    num1 = 0
    temp = df_list[i]
    for j in temp['combination']:
        num1 += len(j)
    print(num1)

In [ ]:
df_list[0]